In [1]:
# Pré processanmento, tirar ruido e informações inconsistentes para fazer a aplicação correta do algoritimo

In [2]:
import pandas as pd

# Excluindo features desbalanceadas e com pouco poder explicativo

In [3]:
# O parâmetro 'enconding' devido as caracteristicas do dataset, que tem varias strings
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')

base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)

base.head()

,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,Golf_3_1.6,privat,Angebot,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
1,A5_Sportback_2.7_Tdi,privat,Angebot,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
2,"Jeep_Grand_Cherokee_""Overland""",privat,Angebot,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
3,GOLF_4_1_4__3TÜRER,privat,Angebot,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,Skoda_Fabia_1.4_TDI_PD_Classic,privat,Angebot,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


In [4]:
# Fazendo um primeiro entendimento das features é possivel descartar varias features
# Intuitivamente é possivel ver de forma clara que varias features não tem relação nenhuma com o nosso target
# Target esse que é o preço

In [5]:
base['name'].value_counts()

Ford_Fiesta                                                        657
BMW_318i                                                           627
Opel_Corsa                                                         622
Volkswagen_Golf_1.4                                                603
BMW_316i                                                           523
                                                                  ... 
Honda_Civic_tauch_moeglich                                           1
Mercedes_Benz_C_200_Komp._TOP_Zustand__Anhaengerkpl._TÜV12/17        1
Porsche_911_993_TURBO_WLS_1_NEUWAGENZUSTAND                          1
MAZDA_6_CD_2.0DI_16V_Sport_Kombi_Exclusiv_GRÜNE_UMWELTPLAKETTE4      1
Volkswagen_Passat_Variant_1.9_TDI_4Motion__standheiz._navi           1
Name: name, Length: 233531, dtype: int64

In [6]:
# Alta variabilidade, logo fica dificil encontrar um padrão com um 'Lenght: 233531'
# Ou seja 233531 que ocorrem pelo menos uma vez, logo a informação é mais ruidosa que preditiva
# Assim fica dificil encontrar um padrão aqui, podendo descartar essa feature

base = base.drop('name', axis = 1)

In [7]:
base['seller'].value_counts()

privat        371525
gewerblich         3
Name: seller, dtype: int64

In [8]:
# Informação desbalanceada, logo é possivel descartar

base = base.drop('seller', axis = 1)

In [9]:
base['offerType'].value_counts()

Angebot    371516
Gesuch         12
Name: offerType, dtype: int64

In [10]:
# Informação desbalanceada, logo é possivel descartar

base = base.drop('offerType', axis = 1)

In [11]:
# O ideal é executar value_counts para todos os atributos e verificar se há esse desbalanceamento
# Para retirar ruídos, aqui pegamos direto os exemplos desbalanceados.

# Valores inconsistentes

In [12]:
# Podem ter ocorrido falhar na hora da extração dos dados, então podemos eliminar dados que fogem da realidade
# Como preço dos carros muito baixos e muito altos
# Poderiamos substituir os registros inconsistentes pela média de todos, porém são muitos registros inconsistentes
# O que poderia ocasionar em uma distorção no resultado então nao vamos substituir pela média
# Logo excluimos os valores extremos que fogem da realidade, enxugando nosso dataset e eliminando ruído.

In [13]:
i1 = base.loc[base.price <= 10]
i1.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
7,0,test,limousine,1980,manuell,50,andere,40000,7,benzin,volkswagen,nein
40,0,test,NaN,1990,NaN,0,corsa,150000,1,benzin,opel,NaN
60,1,control,suv,1994,manuell,286,NaN,150000,11,NaN,sonstige_autos,NaN
91,1,control,limousine,1995,manuell,113,e_klasse,150000,4,diesel,mercedes_benz,nein
115,0,test,NaN,2017,manuell,0,golf,5000,12,benzin,volkswagen,NaN


In [14]:
base = base[base.price > 10]

In [15]:
i2 = base.loc[base.price > 350000]
i2.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1846,579000,control,coupe,1980,manuell,277,andere,20000,12,benzin,bmw,nein
10649,420000,control,coupe,2004,manuell,483,911,50000,4,benzin,porsche,nein
14663,11111111,control,coupe,2003,manuell,64,polo,150000,2,benzin,volkswagen,NaN
16889,1000000,control,kombi,1998,NaN,0,mondeo,150000,0,benzin,ford,ja
20143,1250000,test,coupe,2016,manuell,500,911,5000,3,benzin,porsche,nein


In [16]:
base = base.loc[base.price < 350000]

# Valores Faltantes

## Detectar os parametros que tem dados faltantes, só que simplesmente deletar os registros que tem dados faltantes não é a melhor solução uma vez que se for fazer isso para cada parâmetro irá ter uma redução muito substancial do dataset

## Logo a melhor solução é, para cada feature categórica, ver qual o tipo de cada feature categórica tem mais ocorrências e substituir os dados faltantes pelo tipo com mais ocorrências como será feito abaixo.

In [17]:
base.loc[pd.isnull(base['vehicleType'])].head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
22,2900,test,NaN,2018,manuell,90,meriva,150000,5,benzin,opel,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
31,899,control,NaN,2016,manuell,60,clio,150000,6,benzin,renault,NaN


In [18]:
base['vehicleType'].value_counts() # limousine

limousine     93614
kleinwagen    78014
kombi         65921
bus           29699
cabrio        22509
coupe         18386
suv           14477
andere         3125
Name: vehicleType, dtype: int64

In [19]:
base.loc[pd.isnull(base['gearbox'])].head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
15,450,test,kleinwagen,1910,NaN,0,ka,5000,0,benzin,ford,NaN
16,300,test,NaN,2016,NaN,60,polo,150000,0,benzin,volkswagen,NaN
32,245,test,limousine,1994,NaN,0,golf,150000,2,benzin,volkswagen,nein
37,1500,test,NaN,2016,NaN,0,kangoo,150000,1,diesel,renault,nein
70,1200,test,coupe,2001,NaN,0,astra,150000,0,NaN,opel,NaN


In [20]:
base['gearbox'].value_counts() # manuell

manuell      266547
automatik     75508
Name: gearbox, dtype: int64

In [21]:
base.loc[pd.isnull(base['model'])].head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
1,18300,test,coupe,2011,manuell,190,NaN,125000,5,diesel,audi,ja
83,350,control,kleinwagen,1997,manuell,54,NaN,150000,3,NaN,fiat,ja
139,1450,control,limousine,1992,manuell,136,NaN,150000,0,NaN,audi,nein
156,6799,control,kleinwagen,2009,NaN,60,NaN,20000,5,benzin,volkswagen,nein
165,500,control,kleinwagen,1999,manuell,0,NaN,150000,0,benzin,renault,nein


In [22]:
base['model'].value_counts() # golf

golf               28989
andere             25560
3er                19905
polo               12604
corsa              12149
                   ...  
serie_2                8
rangerover             6
serie_3                3
discovery_sport        1
serie_1                1
Name: model, Length: 251, dtype: int64

In [23]:
base.loc[pd.isnull(base['fuelType'])].head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
13,2500,control,kombi,2004,manuell,131,passat,150000,2,NaN,volkswagen,nein
26,5555,control,NaN,2017,manuell,125,c4,125000,4,NaN,citroen,nein
36,1600,control,andere,1991,manuell,75,kadett,70000,0,NaN,opel,NaN
41,7500,control,limousine,2002,automatik,306,e_klasse,150000,4,NaN,mercedes_benz,NaN


In [24]:
base['fuelType'].value_counts() # benzin

benzin     217582
diesel     106002
lpg          5222
cng           557
hybrid        271
andere        165
elektro       101
Name: fuelType, dtype: int64

In [25]:
base.loc[pd.isnull(base['notRepairedDamage'])].head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,NaN,1993,manuell,0,golf,150000,0,benzin,volkswagen,NaN
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,NaN
8,14500,control,bus,2014,manuell,125,c_max,30000,8,benzin,ford,NaN
9,999,test,kleinwagen,1998,manuell,101,golf,150000,0,NaN,volkswagen,NaN
12,999,control,kombi,1995,manuell,115,passat,150000,11,benzin,volkswagen,NaN


In [26]:
base['notRepairedDamage'].value_counts() # nein

nein    259301
ja       34004
Name: notRepairedDamage, dtype: int64

In [27]:
# Finalmente depois de identificar as features categóricas com dados faltantes e identificar os tipos com mais ocorrências
# trocar os dados faltantes de cada features pelos seus respectivos tipos com mais ocorrências.
# Deixando a base de dados sem dados faltantes

In [28]:
valores = {'vehicleType': 'limousine', 'gearbox' : 'manuell', 'model' : 'golf',
              'fuelType' : 'benzin', 'notRepairedDamage' : 'nein'}

In [29]:
base = base.fillna(value = valores)
base.head()

,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,kilometer,monthOfRegistration,fuelType,brand,notRepairedDamage
0,480,test,limousine,1993,manuell,0,golf,150000,0,benzin,volkswagen,nein
1,18300,test,coupe,2011,manuell,190,golf,125000,5,diesel,audi,ja
2,9800,test,suv,2004,automatik,163,grand,125000,8,diesel,jeep,nein
3,1500,test,kleinwagen,2001,manuell,75,golf,150000,6,benzin,volkswagen,nein
4,3600,test,kleinwagen,2008,manuell,69,fabia,90000,7,diesel,skoda,nein


# Label Encoder - Transformando dados categóricos em númericos

In [30]:
# Primeiro dividi-se o dataset entre o target e os atributos

previsores = base.iloc[:, 1:13].values # Todas as colunas, exceto a 'price'
preco_real = base.iloc[:, 0].values # Coluna 'price'

In [31]:
previsores[0] 
# pode-se inferir que as colunas 0, 1 , 3 , 5 , 8, 9, 10 são categoricas e terão que ser convertidas para númerico

array(['test', 'limousine', 1993, 'manuell', 0, 'golf', 150000, 0,
       'benzin', 'volkswagen', 'nein'], dtype=object)

In [32]:
previsores_pd = base.iloc[:, 1:13]

In [33]:
# TUdo que for string, ou como vemos abaixo 'object', deve ser convertido ao Label Encoder
previsores_pd.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 359291 entries, 0 to 371527
Data columns (total 11 columns):
abtest                 359291 non-null object
vehicleType            359291 non-null object
yearOfRegistration     359291 non-null int64
gearbox                359291 non-null object
powerPS                359291 non-null int64
model                  359291 non-null object
kilometer              359291 non-null int64
monthOfRegistration    359291 non-null int64
fuelType               359291 non-null object
brand                  359291 non-null object
notRepairedDamage      359291 non-null object
dtypes: int64(4), object(7)
memory usage: 32.9+ MB


In [34]:
from sklearn.preprocessing import LabelEncoder
label_encoder_previsores = LabelEncoder()

In [35]:
previsores[:,0] = label_encoder_previsores.fit_transform(previsores[:,0])
previsores[:,1] = label_encoder_previsores.fit_transform(previsores[:,1])
previsores[:,3] = label_encoder_previsores.fit_transform(previsores[:,3])
previsores[:,5] = label_encoder_previsores.fit_transform(previsores[:,5])
previsores[:,8] = label_encoder_previsores.fit_transform(previsores[:,8])
previsores[:,9] = label_encoder_previsores.fit_transform(previsores[:,9])
previsores[:,10] = label_encoder_previsores.fit_transform(previsores[:,10])

# One Hot Encoder - Transformando variaveis numéricas em variaveis dummy

In [40]:
from sklearn.preprocessing import OneHotEncoder

In [39]:
previsores[0:20] 

# Como podemos observar, agora há apenas valores númericos

# Temos que transforma cada feature em combinações dummy como caso do iris no exemplo anterior, para três clases

# iris setosa    0 ---> 1 0 0
# iris virginica 1 ---> 0 1 0
# iris versicolo 2 ---> 0 0 1 

array([[1, 6, 1993, 1, 0, 118, 150000, 0, 1, 38, 1],
       [1, 3, 2011, 1, 190, 118, 125000, 5, 3, 1, 0],
       [1, 7, 2004, 0, 163, 119, 125000, 8, 3, 14, 1],
       [1, 4, 2001, 1, 75, 118, 150000, 6, 1, 38, 1],
       [1, 4, 2008, 1, 69, 103, 90000, 7, 3, 31, 1],
       [1, 6, 1995, 1, 102, 11, 150000, 10, 1, 2, 0],
       [1, 2, 2004, 1, 109, 8, 150000, 8, 1, 25, 1],
       [0, 1, 2014, 1, 125, 61, 30000, 8, 1, 10, 1],
       [1, 4, 1998, 1, 101, 118, 150000, 0, 1, 38, 1],
       [0, 6, 2004, 1, 105, 10, 150000, 12, 1, 19, 1],
       [0, 5, 2005, 1, 140, 171, 150000, 12, 3, 38, 0],
       [0, 5, 1995, 1, 115, 171, 150000, 11, 1, 38, 1],
       [0, 5, 2004, 1, 131, 171, 150000, 2, 1, 38, 1],
       [0, 7, 2011, 1, 190, 161, 70000, 3, 3, 23, 1],
       [1, 4, 1910, 1, 0, 130, 5000, 0, 1, 10, 1],
       [1, 6, 2016, 1, 60, 174, 150000, 0, 1, 38, 1],
       [0, 4, 2004, 0, 75, 228, 150000, 2, 1, 27, 1],
       [1, 1, 2007, 1, 136, 61, 150000, 6, 3, 10, 1],
       [1, 1, 2004, 1, 102,

In [48]:
onehotencoder = OneHotEncoder(categorical_features = [0, 1, 3, 5, 8, 9, 10] )
previsores = onehotencoder.fit_transform(previsores).toarray()

/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/home/jarbasjr/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:451: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
# Esse tipo de recurso quando não existe ordem de importância entre os atributos
# Por exemplo, não existe uma ordem entre os tipos de cambio do veiculo(automatico ou manual)m são apenas diferentes
# Logo a variavel dummy dá essa equanimidade, que com apenas o Label Encoder, poderia dar essa ordem de importancia inexistente, o que com certeza afetaria o resultado final.

In [52]:
previsores.shape

(359291, 344)

# Estrutura da Rede Neural

In [53]:
from keras.models import Sequential
from keras.layers import Dense

In [58]:
regressor = Sequential()

# Mesma regra, (entrada + saida) / 2, nesse caso (344 + 1) / 2 = 172.5, arredondando 173.
regressor.add(Dense(units = 173, activation = 'relu', input_dim = 344 ))
regressor.add(Dense(units = 173, activation = 'relu'))
regressor.add(Dense(units = 1, activation = 'linear'))

regressor.compile(loss = 'mean_absolute_error', optimizer = 'adam', metrics = ['mean_absolute_error'])

regressor.fit(previsores, preco_real, batch_size = 300, epochs = 100)

Epoch 1/100
359291/359291 [==============================] - 4s 12us/step - loss: 3808.4622 - mean_absolute_error: 3808.4622
Epoch 2/100
359291/359291 [==============================] - 4s 11us/step - loss: 3155.7727 - mean_absolute_error: 3155.7727
Epoch 3/100
359291/359291 [==============================] - 4s 11us/step - loss: 2908.7106 - mean_absolute_error: 2908.7106
Epoch 4/100
359291/359291 [==============================] - 4s 12us/step - loss: 2801.5564 - mean_absolute_error: 2801.5564
Epoch 5/100
359291/359291 [==============================] - 4s 12us/step - loss: 2760.4746 - mean_absolute_error: 2760.4746
Epoch 6/100
359291/359291 [==============================] - 4s 11us/step - loss: 2716.8422 - mean_absolute_error: 2716.8422
Epoch 7/100
359291/359291 [==============================] - 4s 12us/step - loss: 2669.1080 - mean_absolute_error: 2669.1080
Epoch 8/100
359291/359291 [==============================] - 4s 12us/step - loss: 2633.8715 - mean_absolute_error: 2633.8715


359291/359291 [==============================] - 4s 12us/step - loss: 2273.4872 - mean_absolute_error: 2273.4872
Epoch 67/100
359291/359291 [==============================] - 4s 12us/step - loss: 2279.1212 - mean_absolute_error: 2279.1212
Epoch 68/100
359291/359291 [==============================] - 4s 12us/step - loss: 2273.4786 - mean_absolute_error: 2273.4786
Epoch 69/100
359291/359291 [==============================] - 5s 14us/step - loss: 2287.4183 - mean_absolute_error: 2287.4183
Epoch 70/100
359291/359291 [==============================] - 4s 12us/step - loss: 2282.4382 - mean_absolute_error: 2282.4382
Epoch 71/100
359291/359291 [==============================] - 4s 12us/step - loss: 2274.1837 - mean_absolute_error: 2274.1837
Epoch 72/100
359291/359291 [==============================] - 4s 12us/step - loss: 2276.1646 - mean_absolute_error: 2276.1646
Epoch 73/100
359291/359291 [==============================] - 4s 11us/step - loss: 2266.4941 - mean_absolute_error: 2266.4941
Epoch

In [59]:
previsoes = regressor.predict(previsores)

In [57]:
# Aqui não separamos em dados de teste e treino, utilizamos toda a base de dados. 

In [ ]:
previsoes.mean()

In [ ]:
preco_real.mean()